In [714]:
import time
from collections import Counter

In this assignment you will implement one or more algorithms for the 2SAT problem. Here are 6 different 2SAT instances.

The file format is as follows. In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file. Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not". For example, the second line of the first data file is "-16808 75250", which indicates the clause $¬x_{16808} ∨ x_{75250}$.

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable. In the box below, enter a 6-bit string, where the ith bit should be 1 if the ith instance is satisfiable, and 0 otherwise. For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

DISCUSSION: This assignment is deliberately open-ended, and you can implement whichever 2SAT algorithm you want. For example, 2SAT reduces to computing the strongly connected components of a suitable graph (with two vertices per variable and two directed edges per clause, you should think through the details). This might be an especially attractive option for those of you who coded up an SCC algorithm in Part 2 of this specialization. Alternatively, you can use Papadimitriou's randomized local search algorithm. (The algorithm from lecture is probably too slow as stated, so you might want to make one or more simple modifications to it --- even if this means breaking the analysis given in lecture --- to ensure that it runs in a reasonable amount of time.) A third approach is via backtracking. In lecture we mentioned this approach only in passing; see Chapter 9 of the Dasgupta-Papadimitriou-Vazirani book, for example, for more details.

In [715]:
def complement_index(u,v,num_var):
    """If index is negative, -x maps it to x+num_var for indexing purpose. Calculate the complements and also
    map it for indexing purpose"""
    if u>0 and v>0:
        u_c = u + num_var
        v_c = v + num_var
    elif u>0 and v<0:
        u_c = u+num_var
        v_c = -v
        v = -v + num_var        
    elif u<0 and v>0:
        v_c = v+num_var
        u_c = -u
        u = -u + num_var
    elif u<0 and v<0:
        u_c = -u
        u = -u + num_var
        v_c = -v
        v = -v + num_var 
    return u,v,u_c,v_c

In [716]:
def read_input(filename):
    with open('week16_file/'+filename) as f:
        first = True
        orig_G = []
        G = {}
        G_rev = {} #G rev = G with all arcs reversed
        for line in f:
            line = line.split() # to deal with blank 
            if line:            # lines (ie skip them)
                if not first:
                    u = int(line[0])
                    v = int(line[1])
                    u,v,u_c,v_c = complement_index(u,v,num_var)
                    
                    #Create implication graph x1 v x2 equals to -x1 -> x2 and -x2 -> x1
                    if u_c not in G:
                        G[u_c] = [(u_c,v)]
                    else:
                        G[u_c].append((u_c,v))
                    
                    if v_c not in G:
                        G[v_c] = [(v_c,u)]
                    else:
                        G[v_c].append((v_c,u))
                    
                    if v not in G_rev:
                        G_rev[v] = [(v,u_c)]
                    else:
                        G_rev[v].append((v,u_c))
                        
                    if u not in G_rev:
                        G_rev[u] = [(u,v_c)]
                    else:
                        G_rev[u].append((u,v_c))
                else:
                    num_var = int(line[0])
                    first = False
    return G,G_rev,num_var

In [717]:
class Graph:
        
    def __init__(self,n,graph_dict=None):
        if graph_dict == None:
            graph_dict = {}
        self.graph_dict = graph_dict
        self.length = n
        self.explored = [0]*(self.length+1)
        self.leader = [0]*(self.length+1)
        self.finishing_time = [0]*(self.length+1)

In [718]:
def DFS_loop(G):
    global t #finishing time in first pass, number of nodes processed so far
    global s #leaders in second pass
    t = 0
    s = None
    n = G.length
    for i in range(n,0,-1):
        if G.explored[i] == False: #if not yet explored
            s = i
            DFS(G,i)

In [719]:
def DFS(G,i):
    global t
    global s
    stack = [i]
    stack2 = [] #to keep track of the finishing time for iterative dfs
    while len(stack):
        v = stack.pop()
        if G.explored[v] == False:
            G.explored[v] = True
            G.leader[v] = s
            stack2.append(v)
            if G.graph_dict.get(v) != None:
                for (v,w) in G.graph_dict[v]:
                    if G.explored[w] == False:
                        stack.append(w)            
    for i in reversed(stack2):
        t += 1
        G.finishing_time[i] = t

In [720]:
def two_SAT_SCC(filename):
    G_dict,G_rev_dict,num_var = read_input(filename)
    n = 2*num_var
    G_rev = Graph(n,G_rev_dict)
    
    #Run DFS loop on G_rev to compute "magical ordering" of nodes
    DFS_loop(G_rev)
    G_new_dict = {}
    
    #Processing nodes in decreasing order of finishing times
    for k,v in G_dict.items():
        new_k = G_rev.finishing_time[k]
        new_v = [(G_rev.finishing_time[i],G_rev.finishing_time[j]) for (i,j) in v]
        G_new_dict[new_k] = new_v
    G_new = Graph(n,G_new_dict)
    
    #Run DFS loop on G to discover the SCC one by one
    DFS_loop(G_new)

    for i in range(num_var):
        #Convert the indexing back to the original graph indexing
        #Check if it is on the same SCC by comparing its leader
        if G_new.leader[G_rev.finishing_time[i]] == G_new.leader[G_rev.finishing_time[i+num_var]]:
            return False
    return True
    

In [721]:
start_time = time.time()
print(two_SAT_SCC('week16_test1.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test2.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test3.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test4.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test5.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))

False
--- 0.009557962417602539 seconds ---
True
--- 0.0014472007751464844 seconds ---
True
--- 0.0011589527130126953 seconds ---
True
--- 0.0011069774627685547 seconds ---
False
--- 0.0011990070343017578 seconds ---


In [722]:
start_time = time.time()
print(two_SAT_SCC('week16_test6.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test7.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_test8.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))

False
--- 2.1071982383728027 seconds ---
True
--- 2.7855238914489746 seconds ---
False
--- 2.8174281120300293 seconds ---


In [711]:
start_time = time.time()
print(two_SAT_SCC('week16_1.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_2.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_3.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))

True
--- 3.237689733505249 seconds ---
False
--- 5.61759877204895 seconds ---
True
--- 13.267502069473267 seconds ---


In [712]:
start_time = time.time()
print(two_SAT_SCC('week16_4.txt')) #True
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_5.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(two_SAT_SCC('week16_6.txt')) #False
print("--- %s seconds ---" % (time.time() - start_time))

True
--- 19.589258909225464 seconds ---
False
--- 24.889580011367798 seconds ---
False
--- 30.36001682281494 seconds ---
